In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [10]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("calculus50.pdf")
docs = loader.load()
print(f"문서의 페이지수: {len(docs)}")

문서의 페이지수: 50


In [11]:
print(docs[10].page_content)

Contents	
ix
	
15.6	 Triple Integrals  1029
	
	
 
Discovery Project  •  Volumes of Hyperspheres  1040
	
15.7	 Triple Integrals in Cylindrical Coordinates  1040
	
	
 
Discovery Project  •  The Intersection of Three Cylinders   1044
	
15.8	 Triple Integrals in Spherical Coordinates  1045
	
	
 
Applied Project  •  Roller Derby  1052
	
15.9	 Change of Variables in Multiple Integrals  1052
	
	
Review  1061
	
Problems Plus  1065
16
16
	
16.1	
Vector Fields  1068
	
16.2	
Line Integrals  1075
	
16.3	
The Fundamental Theorem for Line Integrals  1087
	
16.4	
Green’s Theorem  1096
	
16.5	
Curl and Divergence  1103
	
16.6	
Parametric Surfaces and Their Areas  1111
	
16.7	
Surface Integrals  1122
	
16.8	
Stokes’ Theorem  1134
	
	
 
Writing Project  •  Three Men and Two Theorems  1140
	
16.9	
The Divergence Theorem  1141
	
16.10	 Summary  1147
	
	
Review  1148
	
Problems Plus  1151
17
17
	
17.1	
Second-Order Linear Equations  1154
	
17.2	
Nonhomogeneous Linear Equations  1160
	
17.3	
Applications of

In [12]:
docs[10].__dict__

{'id': None,
 'metadata': {'producer': 'iLovePDF',
  'creator': '',
  'creationdate': '',
  'source': 'calculus50.pdf',
  'file_path': 'calculus50.pdf',
  'total_pages': 50,
  'format': 'PDF 1.7',
  'title': '',
  'author': '',
  'subject': '',
  'keywords': '',
  'moddate': '2025-05-01T06:34:31+00:00',
  'trapped': '',
  'modDate': 'D:20250501063431Z',
  'creationDate': '',
  'page': 10},
 'page_content': 'Contents\t\nix\n\t\n15.6\t Triple Integrals\u2002 1029\n\t\n\t\n\u2003\nDiscovery Project\u2002 •\u2002 Volumes of Hyperspheres\u2002 1040\n\t\n15.7\t Triple Integrals in Cylindrical Coordinates\u2002 1040\n\t\n\t\n\u2003\nDiscovery Project\u2002 •\u2002 The Intersection of Three Cylinders \u2002 1044\n\t\n15.8\t Triple Integrals in Spherical Coordinates\u2002 1045\n\t\n\t\n\u2003\nApplied Project\u2002 •\u2002 Roller Derby\u2002 1052\n\t\n15.9\t Change of Variables in Multiple Integrals\u2002 1052\n\t\n\t\nReview\u2002 1061\n\t\nProblems Plus\u2002 1065\n16\n16\n\t\n16.1\t\nVector 

In [13]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_documents = text_splitter.split_documents(docs)
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 316


In [25]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004"
) 

In [26]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [28]:
for doc in vectorstore.similarity_search("The preprogrammed functions"):
    print(doc.page_content)

Section 1.1    Four Ways to Represent a Function	
11
It’s helpful to think of a function as a machine (see Figure 2). If x is in the domain of 
the function f, then when x enters the machine, it’s accepted as an input and the machine 
produces an output fsxd according to the rule of the function. Thus we can think of the 
domain as the set of all possible inputs and the range as the set of all possible outputs.
The preprogrammed functions in a calculator are good examples of a function as a
machine. For example, the square root key on your calculator computes such a function. 
You press the key labeled s   (or sx ) and enter the input x. If x , 0, then x is not in the 
domain of this function; that is, x is not an acceptable input, and the calculator will indi­
cate an error. If x > 0, then an approximation to sx  will appear in the display. Thus the 
sx  key on your calculator is not quite the same as the exact mathematical function f 
defined by fsxd −sx .
iii
Preface 
xi
To the Stud

In [29]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [30]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("what is preprogrammed functions?")

[Document(id='139fca1a-30bb-46aa-8190-b1158fc5b48b', metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'calculus50.pdf', 'file_path': 'calculus50.pdf', 'total_pages': 50, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-05-01T06:34:31+00:00', 'trapped': '', 'modDate': 'D:20250501063431Z', 'creationDate': '', 'page': 42}, page_content='Section\u20041.1\u2002 \u2002 Four Ways to Represent a Function\t\n11\nIt’s helpful to think of a function as a machine (see Figure 2). If x is in the domain of \nthe function f, then when x enters the machine, it’s accepted as an input and the machine \nproduces an output fsxd according to the rule of the function. Thus we can think of the \ndomain as the set of all possible inputs and the range as the set of all possible outputs.\nThe preprogrammed functions in a calculator are good examples of a function as a'),
 Document(id='a044932c-5a2b-4899-b027-a8d1f1b9a9ed', metadata={'pr

In [36]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
TheoryExplanationPrompt = PromptTemplate.from_template(
    """You are an engineering mathematics theory explanation assistant.
Use the following retrieved context to answer the user’s question.
Answer in Korean.
If you don’t know the answer, simply reply “I don’t know.”
Always include the following three elements in your response:
  1) Definition
  2) Key formulas (in LaTeX)
  3) A brief example

#Context:
{context}

#Question:
{question}

#Answer:"""
)

In [37]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-1.5-pro",  # 또는 사용 가능한 Gemini 모델 아이디(ex: "models/gemini-2.0-flex")
    temperature=0.0
)

In [38]:
# 단계 8: 체인(Chain) 생성
chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | TheoryExplanationPrompt
        | llm
        | StrOutputParser()
)

In [39]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
question = "The preprogrammed functions에 대해서 알려줘"
response = chain.invoke(question)
print(response)

계산기에 미리 프로그램된 함수는 함수를 이해하는 좋은 예시입니다.

1. **정의:** 함수는 각 입력값에 대해 정확히 하나의 출력값을 할당하는 규칙입니다. 계산기에서 미리 프로그램된 함수는 특정 입력값을 받아 미리 정의된 연산을 수행하고 그 결과를 출력하는 내장 함수입니다.  입력값의 집합을 정의역, 출력값의 집합을 치역이라고 합니다.

2. **주요 공식:** 함수는 일반적으로 $f(x) = y$ 와 같이 표현됩니다. 여기서 $x$는 입력값, $f$는 함수, $y$는 출력값입니다.  계산기의 제곱근 함수는 $f(x) = \sqrt{x}$로 표현할 수 있습니다.

3. **예시:** 계산기에서 제곱근 함수(√)를 생각해 봅시다.  입력값 9를 입력하면, 계산기는 $\sqrt{9}=3$ 을 계산하여 출력값 3을 보여줍니다. 이 경우 입력값 9는 함수의 정의역에 속하고, 출력값 3은 함수의 치역에 속합니다. 만약 음수를 입력하면, 제곱근 함수는 실수 범위에서 정의되지 않으므로 에러가 발생합니다.  즉, 음수는 이 함수의 정의역에 속하지 않습니다.
